Imports

In [19]:
import pandas as pd
import numpy as np
import torch, transformers
import gzip, json

## Data pipeline

Specify paths

In [20]:
# Modify this to wherever you locally downloaded the data
data_base_path = './data/newsroom-release/release/'

train_path = data_base_path + 'train.jsonl.gz'
validation_path = data_base_path + 'dev.jsonl.gz'
test_path = data_base_path + 'dev.jsonl.gz'

Create dataset

In [42]:
class NewsroomDataset(torch.utils.data.Dataset):
    def __init__(self, batch_size, path):
        self.batch_size = batch_size
        self.file = gzip.open(path)
        
    def __len__(self):
        return self.batch_size

    def __getitem__(self, idx):
        return self.load_data()

    
    def load_data(self):
        data = []
        
        for ln in self.file:
            if len(data) < self.batch_size:
                obj = json.loads(ln)
                data.append(obj)
            else:
                return data

Dataset is too big to load to memory - create minibatches and parallelize loading with DataLoader

In [59]:
train_dset = NewsroomDataset(100, train_path)
trainloader = torch.utils.data.DataLoader(train_dset)    

Get batch for visualization

In [57]:
train_df = pd.DataFrame(next(iter(trainloader)))
train_df

0


,url,archive,title,date,text,summary,compression,coverage,density,compression_bin,coverage_bin,density_bin
0,[http://abcnews.go.com/2020/video/flds-controv...,[http://web.archive.org/web/20150511002840id_/...,[Who Are the FLDS? Controversial Community Exp...,[20150511002840],[Now Playing: Great Plains Prepare for Potenti...,[Many questions surround the followers of Warr...,"[tensor(12.2105, dtype=torch.float64)]","[tensor(0.3684, dtype=torch.float64)]","[tensor(0.3684, dtype=torch.float64)]",[low],[low],[abstractive]
1,[http://fortune.com/2015/05/15/temp-jobs-perma...,[http://web.archive.org/web/20150515174943id_/...,[Temp jobs: Will they count against you in a j...,[20150515174943],[Dear Annie: I got laid off from my accounting...,[About half of employers say a steady history ...,"[tensor(36.7778, dtype=torch.float64)]","[tensor(0.8519, dtype=torch.float64)]","[tensor(1.6667, dtype=torch.float64)]",[high],[medium],[mixed]
2,[http://www.9news.com.au/World/2015/05/15/11/1...,[http://web.archive.org/web/20150516095250id_/...,[Family with 12 sons welcome 13th child - and ...,[20150516095250],[Kateri Schwandt with her 13th and newest son....,[A US family with 12 sons have added a 13th ch...,"[tensor(10.5833, dtype=torch.float64)]","[tensor(1., dtype=torch.float64)]","[tensor(24., dtype=torch.float64)]",[low],[high],[extractive]
3,[http://fortune.com/2012/06/26/venture-capital...,[http://web.archive.org/web/20150521050847id_/...,[Venture capital deals],[20150521050847],"[FastPay, a Los Angeles-based finance platform...","[FastPay, a Los Angeles-based finance platform...","[tensor(5.5385, dtype=torch.float64)]","[tensor(0.9359, dtype=torch.float64)]","[tensor(19.3205, dtype=torch.float64)]",[low],[medium],[extractive]
4,[http://fortune.com/2015/05/22/apple-tv-delayed/],[http://web.archive.org/web/20150523214323id_/...,[Apple's TV Service Might Be Delayed],[20150523214323],[Apple’s long quest to get a slice of the tele...,[Another setback in Apple's plan to deliver yo...,"[tensor(18.4545, dtype=torch.float64)]","[tensor(0.4545, dtype=torch.float64)]","[tensor(0.6364, dtype=torch.float64)]",[medium],[low],[abstractive]
...,...,...,...,...,...,...,...,...,...,...,...,...
95,[http://www.people.com/article/8-dead-houston-...,[http://web.archive.org/web/20150811010918id_/...,"[8 Bodies Discovered, Suspect in Custody, Foll...",[20150811010918],[08/09/2015 AT 01:05 PM EDT\n\nA 49-year-old m...,[A motive or causes of death for the deceased ...,"[tensor(20.7692, dtype=torch.float64)]","[tensor(0.9231, dtype=torch.float64)]","[tensor(3.6923, dtype=torch.float64)]",[medium],[medium],[mixed]
96,[http://www.bostonglobe.com/business/2015/08/1...,[http://web.archive.org/web/20150815050055id_/...,[Apps step up to help delayed travelers],[20150815050055],"[NEW YORK — Ricardo Vice Santos, chief executi...","[Ricardo Vice Santos, chief executive of an ap...","[tensor(18.5172, dtype=torch.float64)]","[tensor(1., dtype=torch.float64)]","[tensor(19.4828, dtype=torch.float64)]",[medium],[high],[extractive]
97,[http://www.bostonglobe.com/business/2015/08/1...,[http://web.archive.org/web/20150815050115id_/...,[Drones getting in the way of emergency respon...,[20150815050115],[SAN DIEGO — As Jason Thrasher lowered his hel...,[As Jason Thrasher lowered his helicopter to a...,"[tensor(18.6452, dtype=torch.float64)]","[tensor(1., dtype=torch.float64)]","[tensor(15.1290, dtype=torch.float64)]",[medium],[high],[extractive]
98,[http://www.theguardian.com/artanddesign/2011/...,[http://web.archive.org/web/20150816041702id_/...,[Tom Lubbock obituary],[20150816041702],"[Until the last months of 2010, Tom Lubbock, w...","[Collagist, illustrator and chief art critic o...","[tensor(143.8000, dtype=torch.float64)]","[tensor(0.9000, dtype=torch.float64)]","[tensor(3.9000, dtype=torch.float64)]",[high],[medium],[mixed]


Quick summary

In [64]:
max(train_df.summary, key=len)

["Many questions surround the followers of Warren Jeffs's Fundamentalist Church of Jesus Christ of Latter Day Saints."]